# Getting Started with Prompt Engineering
by DAIR.AI | Elvis Saravia


This notebook contains examples and exercises to learning about prompt engineering.

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples. I am using the default settings `temperature=0.7` and `top-p=1`

---

## 1. Prompt Engineering Basics

Objectives
- Load the libraries
- Review the format
- Cover basic prompts
- Review common use cases

Below we are loading the necessary libraries, utilities, and configurations.

In [17]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv

In [18]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

Load environment variables. You can use anything you like but I used `python-dotenv`. Just create a `.env` file with your `OPENAI_API_KEY` then load it.

In [19]:
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")
# os.environ["SERPER_API_KEY"] = os.getenv("SERPER_API_KEY")

In [75]:
from langchain.llms import OpenAI


def set_open_params(
    model="text-davinci-003",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

# def get_completion(params, prompt):
#     """ GET completion from openai api"""

#     response = openai.Completion.create(
#         engine = params['model'],
#         prompt = prompt,
#         temperature = params['temperature'],
#         max_tokens = params['max_tokens'],
#         top_p = params['top_p'],
#         frequency_penalty = params['frequency_penalty'],
#         presence_penalty = params['presence_penalty'],
#     )
#     return response

#####
# - I suppose the above functions were needed back in the day to wrap the more complex calling; but now since it uses clients and langchain
# - It doesn't really need the above. We can just instantiate a language model with parameters and start calling against it.



def get_completion_llm(params, prompt):
    """ GET completion from openai api using langchain"""
    llm = OpenAI(
        model_name = params['model'], 
        openai_api_key=openai_api_key,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty']
    )
    response = llm(prompt)
    return response




### Parameters explained
- temperature=0.7 - Sets the temperature for the AI's responses, which influences creativity and randomness. Higher values generate more creative and less predictable responses. The default is 0.7.
- max_tokens=256 - Determines the maximum length of the response in tokens (words and characters). The default is 256.
- top_p=1 - Controls the diversity of the responses. A value of 1 means that no sampling is done (the model considers all possible tokens at each step). The default is 1.

#### `frequency_penalty`
- **Purpose**: The `frequency_penalty` parameter influences the likelihood of the model repeating the same line of text or similar text within a single output. 
- **How It Works**: 
  - A positive value (greater than 0) discourages repetition – the higher the value, the less likely the model is to repeat the same words and phrases. 
  - A value of 0 means there is no penalty for repetition, so the model will behave naturally based on how it was trained.
  - Negative values (less than 0) are not typically used and may lead to unexpected behavior.
- **Use Cases**: This parameter is particularly useful when you need the model to generate diverse and varied content, avoiding redundancy. It's beneficial in creative writing, content generation, or any application where you want to minimize repetitiveness.

#### `presence_penalty`
- **Purpose**: The `presence_penalty` parameter affects how likely the model is to bring up new topics or switch contexts within the output.
- **How It Works**: 
  - A positive value (greater than 0) encourages the model to introduce new topics, leading to outputs that might explore new ideas or switch topics more frequently.
  - A value of 0 means there is no alteration to the model's natural inclination to change topics.
  - As with `frequency_penalty`, negative values are typically not used and can result in unpredictable behavior.
- **Use Cases**: This parameter is useful when you want the model to explore or introduce new concepts rather than focusing solely on the concepts already mentioned. It can enhance creativity and breadth in applications like brainstorming, ideation, or when you're seeking a wide range of suggestions or ideas.

### Balancing the Parameters
Both of these parameters are about fine-tuning the AI's output to suit specific needs. They should be balanced and adjusted based on the context and desired output. Excessive penalties might make the text too divergent or incoherent, while too low penalties might not effectively address issues like repetition or lack of novelty. Experimentation with different values in the context of your specific use case is often necessary to find the right balance.


Basic prompt example:

In [ ]:
# Reconstructing the get_completion model to instantiating a LLM instead;



In [76]:
# basic example
params = set_open_params()

# prompt = "The sky is"

# response = get_completion(params, prompt)


from langchain.llms import OpenAI
llm = OpenAI(model_name="text-davinci-003", openai_api_key=openai_api_key)

# I like to use three double quotation marks for my prompts because it's easier to read
prompt = """
Bow wow wow yipee yo yipee --?
# """

response = get_completion_llm(params, prompt)
print(response)
print(type(response))


 Bow wow wow yipee yo yipee yay!
<class 'str'>


In [29]:
# response.choices[0].text # outdated, is now text in text out

Try with different temperature to compare results:

In [36]:
params = set_open_params(temperature=1)
prompt = "The sky is"
response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)

 clear
The sky is clear and blue. The sun is shining and it's a beautiful, warm day. There are no clouds in sight, and the sky is a deep, vivid blue.

### 1.1 Text Summarization

In [37]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence like I am 5:"""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)



Antibiotics help your body fight off bacterial infections by killing or stopping the germs from making more germs.

Exercise: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [38]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)


 Mice

Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

Exercise: Edit prompt and get the model to respond that it isn't sure about the answer. 

### 1.3 Text Classification

In [44]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was just ok.

Sentiment:"""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)

 Neutral

Exercise: Modify the prompt to instruct the model to provide an explanation to the answer selected. 

### 1.4 Role Playing

In [45]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)

 Sure! A black hole is an object with a gravitational pull so strong that nothing, not even light, can escape from it. Black holes are created when a star dies and collapses in on itself. The collapse is so powerful that it creates a singularity, a point of infinite density. This singularity has an incredibly strong gravitational pull, which is what creates the black hole.

Exercise: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [46]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)



SELECT students.StudentId, students.StudentName 
FROM students
INNER JOIN departments 
ON students.DepartmentId = departments.DepartmentId
WHERE departments.DepartmentName = 'Computer Science';

### 1.6 Reasoning

In [82]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage


prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1, 9. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""


chat = ChatOpenAI(model='gpt-4-0613', openai_api_key=openai_api_key, model_name="gpt-4-0613")

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)
# Odd numbers: 15, 5, 13, 7, 1 Total: 41 41 is an odd number.
# Odd numbers: 15, 5, 13, 7, 1, 9 Sum of odd numbers: 40 Result: 40 is an even number.?? <<< the thing is wrong

response = chat([
        SystemMessage(content="You are a logical math teacher, please explain math problems step by step but keep the theory short like I am 5. Please use Markedown in your response"),
        HumanMessage(content=prompt)
    ])

In [84]:
print(response.content)

Alright, let's break this down:

Step 1: Identify the odd numbers. Odd numbers are numbers that cannot be divided evenly by 2. In this group, the odd numbers are 15, 5, 13, 7, 1, and 9. 

Step 2: Add the odd numbers. To do this, simply add the numbers together:
15 + 5 + 13 + 7 + 1 + 9.

Step 3: Add the numbers together. Let's start with the first two numbers:
15 + 5 = 20

Then add the next number:
20 + 13 = 33

And the next:
33 + 7 = 40

And then:
40 + 1 = 41

Finally, add the last number:
41 + 9 = 50

So, the sum of the odd numbers is 50.

Step 4: Determine if the result is odd or even. An even number is any number that can be divided by 2 with no remainder. 50 divided by 2 is 25 with no remainder, so 50 is an even number.

So, the odd numbers in this group do indeed add up to an even number.


## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

In [85]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)

 The answer is True.

### 2.3 Chain-of-Thought (CoT) Prompting

In [87]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1, 9. 
A:"""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)

 Adding all the odd numbers (15, 5, 13, 7, 1, 9) gives 50. The answer is True.

### 2.4 Zero-shot CoT

In [89]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

response = get_completion_llm(params, prompt)
IPython.display.Markdown(response)

 You started with 10 apples, gave 2 away to the neighbor and 2 to the repairman. That leaves you with 6 apples. Then, you bought 5 more, which makes 11. Finally, you ate 1 apple, so you are left with 10 apples.

### 2.5 Self-Consistency
As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here. 

### 2.6 Generate Knowledge Prompting

As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here. 

### 2.6 PAL - Code as Reasoning

We are developing a simple application that's able to reason about the question being asked through code. 

Specifically, the application takes in some data and answers a question about the data input. The prompt includes a few exemplars which are adopted from [here](https://github.com/reasoning-machines/pal/blob/main/pal/prompt/penguin_prompt.py).  

In [90]:
# lm instance
llm = OpenAI(model_name='text-davinci-003', temperature=0)

In [91]:
question = "Which is the oldest penguin?"

In [92]:
PENGUIN_PROMPT = '''
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm. 
We now add a penguin to the table:
James, 12, 90, 12
How many penguins are less than 8 years old?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Add penguin James.
penguins.append(('James', 12, 90, 12))
# Find penguins under 8 years old.
penguins_under_8_years_old = [penguin for penguin in penguins if penguin[1] < 8]
# Count number of penguins under 8.
num_penguin_under_8 = len(penguins_under_8_years_old)
answer = num_penguin_under_8
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm.
Which is the youngest penguin?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins = sorted(penguins, key=lambda x: x[1])
# Get the youngest penguin's name.
youngest_penguin_name = penguins[0][0]
answer = youngest_penguin_name
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm.
What is the name of the second penguin sorted by alphabetic order?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort penguins by alphabetic order.
penguins_alphabetic = sorted(penguins, key=lambda x: x[0])
# Get the second penguin sorted by alphabetic order.
second_penguin_name = penguins_alphabetic[1][0]
answer = second_penguin_name
"""
{question}
"""
'''.strip() + '\n'

Now that we have the prompt and question. We can send it to the model. It should output the steps, in code, needed to get the solution to the answer.

In [107]:
question = "Can you sort the penguins in ages in a list"
llm_out = llm(PENGUIN_PROMPT.format(question=question))
print(llm_out)

# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins_sorted_by_age = sorted(penguins, key=lambda x: x[1])
answer = penguins_sorted_by_age


In [108]:
exec(llm_out)
print(answer)

[('Bernard', 5, 80, 13), ('Louis', 7, 50, 11), ('Gwen', 8, 70, 15), ('Vincent', 9, 60, 11)]


That's the correct answer! Vincent is the oldest penguin. 

In [112]:
question = "Who is the second oldest penguin?"
response = chat([
        SystemMessage(content="You are to use the provided python code and return me the answer / outcome"),
        HumanMessage(content=PENGUIN_PROMPT.format(question=question))
    ])


In [113]:
print(response.content)

# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins = sorted(penguins, key=lambda x: x[1], reverse=True)
# Get the second oldest penguin's name.
second_oldest_penguin_name = penguins[1][0]
answer = second_oldest_penguin_name
# The answer is 'Gwen'.


Exercise: Try a different question and see what's the result.

---

# 3. Tools and Applications

Objective:

- Demonstrate how to use LangChain to demonstrate simple applications using prompting techniques and LLMs

### 3.1 LLMs & External Tools

Example adopted from the [LangChain documentation](https://langchain.readthedocs.io/en/latest/modules/agents/getting_started.html).

In [119]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
serpapi_api_key=os.getenv("SERP_API_KEY", "e7584e546ad3b0f70e90e89fe80b170da724d8ba7e3cb090c6da1346f77c196c")


In [120]:
llm = OpenAI(temperature=0)
toolkit = load_tools(["serpapi", 'llm-math'], llm=llm, serpapi_api_key=serpapi_api_key)

agent = initialize_agent(toolkit, llm, agent="zero-shot-react-description", verbose=True)

In [122]:
# run the agent
agent.run("What happened to Olivia Wilde and her boyfriend Jason?")

# This is insanely cool!!!



> Entering new AgentExecutor chain...
 I should search for recent news about them.
Action: Search
Action Input: Olivia Wilde and Jason news
Observation: ['News of the couple\'s split first broke in November 2020, when PEOPLE exclusively reported that Sudeikis and Wilde had ended their engagement of seven years. " ...', 'Olivia Wilde and Jason Sudeikis reach an agreement in the yearlong custody battle: Sudeikis will pay upwards of $27000 of child support.', 'According to court documents obtained by The Daily Mail, Olivia Wilde and Jason Sudeikis will share joint custody of their son Otis and ...', "One of Hollywood's most low-key, cute couples, Olivia Wilde and Jason Sudeikis, reportedly quietly ended their relationship earlier this year after nine ...", 'Olivia Wilde and Jason Sudeikis called it quits in November 2020, but the pair have weathered their share of drama in the years since. Nearly ...', 'Olivia Wilde and Jason Sudeikis have scored a big win in their legal battle with the

'Olivia Wilde and Jason Sudeikis split in November 2020 and are now in a custody battle.'

### 3.2 Data-Augmented Generation

First, we need to download the data we want to use as source to augment generation.

Code example adopted from [LangChain Documentation](https://langchain.readthedocs.io/en/latest/modules/chains/combine_docs_examples/qa_with_sources.html). We are only using the examples for educational purposes.

Prepare the data first:

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [26]:
with open('./state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [27]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [28]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

Let's quickly test it:

In [29]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [30]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 30-pl'}

Let's try a question with a custom prompt:

In [31]:
template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Spanish.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN SPANISH:"""

# create a prompt template
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

# query 
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "What did the president say about Justice Breyer?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nEl Presidente no dijo nada acerca de la Justicia Breyer.\n\nFUENTES: 30, 31, 33'}

Exercise: Try using a different dataset from the internet and try different prompt, including all the techniques you learned in the lecture.